# Portfolio: `MyCoinbase`

The user portfolio is an object that is kept apart from the definition of objects related to the cryprocurrency portfolio provider (eg. Coinbase API). 

This is done for keeping a level of separation between the two layers, allowing future developement (eg. the implementation of other crypto trading services) and providing methods that are useful specifically at this level.


In [1]:
from surfingcrypto import Config
from surfingcrypto.portfolio import MyCoinbase

In [2]:
parent="/Users/giorgiocaizzi/Documents/GitHub/surfingcrypto/"
c=Config(parent+"config")

## `MyCoinbase` object

The only implemented crypto trading platform is Coinbase and `MyCoinbase` is the class that under the hood of the `Portfolio` object handles the Coinbase user-profile, that is made of the client methods - inherited from the Coinbase API client wrapper object `CB` - plus some additional methods that are used for facilitating the user's sessions.

### Get active accounts

Basic `MyCoinbase` initialization offers loading all accounts that can be considered `active`, a.k.a. that have a `balance`> 0.00 EUR or USD.

In [3]:
cb=MyCoinbase(configuration=c)
print(cb)

MyCoinbase( isHistoric:False, last updated:2022-02-28 22:30:11.085418, N_accounts:6)


In [4]:
print(cb.mycoinbase_report())

AAVE : EUR 125.72
SUSHI : EUR 65.15
MATIC : EUR 360.59
UNI : EUR 120.20
ADA : EUR 329.80
USDC : EUR 0.01
---
Portfolio: EUR 1001.47


### Get all accounts with transactions.

This is the `isHistoric` mode of MyCoinbase, in which the `accounts` represents all accounts that has ever had a transaction. These accounts can either be fetched all from Coinbase API or locally cached.

If the local cached account file is older than 7 days, it is automatically updated. Otherwise, accounts are loaded from the ones specified in dump file.

There is a huge difference in time of execution, loading all accounts that have transactions takes about 1m 20s versus about 12.7s to load accounts from a local dump file.

In [5]:
cb=MyCoinbase(
    active_accounts=False,
    configuration=c)
print(cb)

MyCoinbase( isHistoric:True, last updated:2022-02-27 11:23:57, N_accounts:30)


Obviously the result of `mycoinbase_report()` won't change as it ignores accounts that have `balance`==0.00, but the length of the `accounts` attribute has more accounts than when initalized for only the active ones.

In [6]:
print(cb.mycoinbase_report())

AAVE : EUR 125.72
SUSHI : EUR 65.15
MATIC : EUR 360.59
UNI : EUR 120.20
ADA : EUR 329.80
USDC : EUR 0.01
---
Portfolio: EUR 1001.47


### Transaction history

Get the full transaction history via the `TransactionsHistory` class.

In [7]:
cb.get_history()

In [8]:
cb.history

TransactionsHistory(Processed transactions:177, Unhandled:0, Errors:9 )

In [9]:
cb.history.df.head()

,type,amount,symbol,native_amount,nat_symbol,total,subtotal,total_fee,spot_price,trade_id
datetime,,,,,,,,,,
2022-01-04T10:46:31Z,send,-1.511428,SOL,-226.08,EUR,NaN,NaN,NaN,149.580369,None
2022-01-04T10:44:49Z,trade,1.511428,SOL,225.73,EUR,NaN,NaN,NaN,149.348800,5851e642-8786-585b-a5d7-8546c8c16129
2022-01-02T23:24:25Z,send,-1.026156,SOL,-159.04,EUR,NaN,NaN,NaN,154.986191,None
2021-11-29T10:47:17Z,trade,1.026156,SOL,186.85,EUR,NaN,NaN,NaN,182.087335,18160e53-8019-561c-a812-efd59cc7a354
2021-08-07T14:25:58Z,trade,-2737.634483,ACH,-299.46,EUR,NaN,NaN,NaN,0.109386,4dbcbf9c-cd0d-56cb-8e25-f1da9a64558f


In [10]:
cb.history.unhandled_trans

[]

In [11]:
cb.history.error_log[0]

{'transaction_type': 'buy',
 'account_id': 'af38031b-b9e5-5825-bb3e-77ea783ac192',
 'transaction_id': '22d9ed43-e213-5726-a127-6f80dbb8b3d1',
 'info': {'amount': -200.0, 'symbol': 'EUR', 'date': '2021-04-27T06:07:25Z'},
 'error_log': coinbase.wallet.error.NotFoundError(<Response [404]>,
                                     'not_found',
                                     'Not found',
                                     [{'id': 'not_found',
                                       'message': 'Not found'}])}